In [1]:
import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [3]:
df_temp = pd.read_csv("https://datahub.io/core/global-temp/r/0.csv")

df_temp.head()

,Source,Year,Mean
0,gcag,1850,-0.4177
1,gcag,1851,-0.2333
2,gcag,1852,-0.2294
3,gcag,1853,-0.2704
4,gcag,1854,-0.2915


In [4]:
import os
import dotenv

dotenv.load_dotenv()

chat = ChatOpenAI(api_key=os.getenv("openaiKey"))

agent_temp = create_pandas_dataframe_agent(
    chat,
    df_temp,
    verbose=True,
    agent_type='tool-calling'

)

In [5]:
resposta = agent_temp.invoke({"input":"qual a temperatura média global do ultimo ano?"})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Year'] == df['Year'].max()]['Mean'].values[0]"}`


1.1755A temperatura média global do último ano é de 1.1755.

> Finished chain.


In [3]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain_openai import ChatOpenAI
import os
import dotenv

dotenv.load_dotenv()

True

In [4]:
db_sakila = SQLDatabase.from_uri("sqlite:///sakila_master.db")

chat = ChatOpenAI(api_key=os.getenv("openaiKey"))

agent_executor_sakila = create_sql_agent(
    chat,
    db=db_sakila,
    agent_type="tool-calling",
    verbose=True
)

In [6]:
response = agent_executor_sakila.invoke({"input":"What are the most rented movies??"})
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


actor, address, category, city, country, customer, film, film_actor, film_category, film_text, inventory, language, payment, rental, staff, store
Invoking: `sql_db_schema` with `{'table_names': 'film'}`



CREATE TABLE film (
	film_id INTEGER NOT NULL, 
	title VARCHAR(255) NOT NULL, 
	description TEXT DEFAULT NULL, 
	release_year VARCHAR(4) DEFAULT NULL, 
	language_id SMALLINT NOT NULL, 
	original_language_id SMALLINT DEFAULT NULL, 
	rental_duration SMALLINT DEFAULT 3 NOT NULL, 
	rental_rate DECIMAL(4, 2) DEFAULT 4.99 NOT NULL, 
	length SMALLINT DEFAULT NULL, 
	replacement_cost DECIMAL(5, 2) DEFAULT 19.99 NOT NULL, 
	rating VARCHAR(10) DEFAULT 'G', 
	special_features VARCHAR(100) DEFAULT NULL, 
	last_update TIMESTAMP NOT NULL, 
	PRIMARY KEY (film_id), 
	CONSTRAINT fk_film_language FOREIGN KEY(language_id) REFERENCES language (language_id), 
	CONSTRAINT fk_film_language_original FOREIGN KEY(original

/home/lucas/Área de Trabalho/llm_curso/.venv/lib/python3.12/site-packages/langchain_community/utilities/sql_database.py:313: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
/home/lucas/Área de Trabalho/llm_curso/.venv/lib/python3.12/site-packages/langchain_community/utilities/sql_database.py:325: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables



Invoking: `sql_db_query` with `{'query': 'SELECT title, rental_duration, rental_rate, replacement_cost FROM film ORDER BY rental_duration DESC LIMIT 10'}`


[('ADAPTATION HOLES', 7, 2.99, 18.99), ('ANONYMOUS HUMAN', 7, 0.99, 12.99), ('ARGONAUTS TOWN', 7, 0.99, 12.99), ('BIKINI BORROWERS', 7, 4.99, 26.99), ('BLACKOUT PRIVATE', 7, 2.99, 12.99), ('BLANKET BEVERLY', 7, 2.99, 21.99), ('BOILED DARES', 7, 4.99, 13.99), ('BOONDOCK BALLROOM', 7, 0.99, 14.99), ('BORN SPINAL', 7, 4.99, 17.99), ('BORROWERS BEDAZZLED', 7, 0.99, 22.99)]The movies with the longest rental duration are:

1. Title: ADAPTATION HOLES, Rental Duration: 7 days, Rental Rate: $2.99, Replacement Cost: $18.99
2. Title: ANONYMOUS HUMAN, Rental Duration: 7 days, Rental Rate: $0.99, Replacement Cost: $12.99
3. Title: ARGONAUTS TOWN, Rental Duration: 7 days, Rental Rate: $0.99, Replacement Cost: $12.99
4. Title: BIKINI BORROWERS, Rental Duration: 7 days, Rental Rate: $4.99, Replacement Cost: $26.99
5. Title: BLACKOUT PRIVATE, Rent

In [1]:
from langchain_community.agent_toolkits.sql.base import SQLDatabaseToolkit,create_sql_agent

In [8]:
sql = SQLDatabaseToolkit(db=db_sakila,llm=chat)

sql.get_tools()

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7b52dd7b00e0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7b52dd7b00e0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7b52dd7b00e0>),
 QuerySQLCheckerTool(description='Use this tool to double check